## 1.3 캔들 사용
인공지능으로 숫자를 예측하는 실습을 하면서 파이썬 텍스트 모드와 파이썬 노트북 모드에서 케라스 사용법을 익힙시다.

### 1.3.0 캔들 테스트

In [3]:
:dep candle-core
:dep candle-nn


### 1.3.2 텍스트 모드 실습

1. 실습 진행에서 가장 먼저 할 일은 캔들 패키지 및 다른 필요한 패키지의 관련 기능을 임포트하는 겁니다.

In [35]:
use candle_core::{DType, Device, Result, Tensor};
use candle_nn::{linear, AdamW, Linear, Module, Optimizer, ParamsAdamW, VarBuilder, VarMap};

2. 학습 및 평가에 사용할 전체 데이터를 지정합니다. 

In [110]:
fn gen_xy_data(x: Vec<f32>) -> Result<(Tensor, Tensor)> {
    let x = Tensor::new(x, &Device::Cpu)?; //vec1
    let y = ((x.clone() * 2.0)? + 1.0)?; // Note: use of clone(), ? 

    println!("[gen_data]");
    println!("x: {:?}", x); // show the property of x
    println!("x.to_vec1::<f32>()?: {:?}", x.to_vec1::<f32>()?);
    println!("y.to_vec1::<f32>()?: {:?}", y.to_vec1::<f32>()?); // Note: use of <f32>
    println!();
    Ok((x, y))
}

In [111]:
fn print_model(model: &Linear) -> Result<()> {
    println!("model: {:?}", model);    
    println!("model.weight(): {:?}", model.weight());
    println!("model.weight().to_vec2::<f32>()?: {:?}", model.weight().to_vec2::<f32>()?);
    println!("model.bias(): {:?}", model.bias()); //unwrap  
    println!("model.bias().unwrap(): {:?}", model.bias().unwrap()); //unwrap 
    println!("model.bias().unwrap().to_vec1::<f32>()?: {:?}", model.bias().unwrap().to_vec1::<f32>()?); //unwrap  
    Ok(())
}

In [112]:

fn main() -> Result<()> {
    let (x, y) = gen_xy_data(vec![0_f32, 1.])?;
    
    println!("[Define and Test Model]");
    let varmap = VarMap::new();
    let vb = VarBuilder::from_varmap(&varmap, DType::F32, &Device::Cpu);
    let model = linear(1, 1, vb.pp("linear"))?;
    print_model(&model)?;

    println!("x.dims1(): {:?}", x.dims1()?);
    let x = x.reshape((x.dims1()?,1))?; // reshpe to provide for model
    let y = y.reshape((y.dims1()?,1))?; 
    let y_pred = model.forward(&x)?;
    println!("{:?}", y_pred);
    println!("y_pred.to_vec2::<f32>()?: {:?}", y_pred.to_vec2::<f32>()?);
    println!();

    println!("[Training]");
    let params = ParamsAdamW {
        lr: 0.1,
        ..Default::default()
    };
    let mut opt = AdamW::new(varmap.all_vars(), params)?;

    for step in 0..=1000 {
        let y_pred = model.forward(&x)?;
        let loss = y_pred.sub(&y)?.sqr()?.sum_all()?;
        opt.backward_step(&loss)?;
        if step % 100 == 0 {
            println!("{step} {}", loss.to_vec0::<f32>()?);
        }
    }
    print_model(&model);

    Ok(())
}

main()

[gen_data]
x: Tensor[0, 1; f32]
x.to_vec1::<f32>()?: [0.0, 1.0]
y.to_vec1::<f32>()?: [1.0, 3.0]

[Define and Test Model]
model: Linear { weight: Tensor[dims 1, 1; f32], bias: Some(Tensor[-0.3868711; f32]) }
model.weight(): Tensor[dims 1, 1; f32]
model.weight().to_vec2::<f32>()?: [[0.6914926]]
model.bias(): Some(Tensor[-0.3868711; f32])
model.bias().unwrap(): Tensor[-0.3868711; f32]
model.bias().unwrap().to_vec1::<f32>()?: [-0.3868711]
x.dims1(): 2
Tensor[dims 2, 1; f32]
y_pred.to_vec2::<f32>()?: [[-0.3868711], [0.30462152]]

[Training]
0 9.1884775
100 0.00014220166
200 0.000076096876
300 0.00004889496
400 0.000034905435
500 0.000026533673
600 0.000020980979
700 0.000017043369
800 0.000014116827
900 0.000011864497
1000 0.000010090471
model: Linear { weight: Tensor[dims 1, 1; f32], bias: Some(Tensor[1.0007482; f32]) }
model.weight(): Tensor[dims 1, 1; f32]
model.weight().to_vec2::<f32>()?: [[1.9961674]]
model.bias(): Some(Tensor[1.0007482; f32])
model.bias().unwrap(): Tensor[1.0007482; f

Ok(())